# Transfer-Learning

Download images and store in resource folder (/res)
- https://downloads.codingcoursestv.eu/037%20-%20neuronale%20netze/PetImages.zip



# Prepare Data

In [ ]:
# imports
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
import numpy as np
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
# local imports
import plai.workspace.init # source setup.bash


In [ ]:
# Print sample image of dataset
ws_root = plai.workspace.init.get_ws_path()
dataset_path = os.path.join(ws_root, "res", "data", "PetImages")
os.listdir( os.path.join(dataset_path, "Cat") )
Image.open( os.path.join(dataset_path, "Cat", "9240.jpg") )

In [ ]:
# Prepare data
# @todo Use this ImageDataGenerator for large datasets (e.g. above RAM storage)
gen = ImageDataGenerator()

train_generator = gen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
# @brief
def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    # Prepare images (resize)
    # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html
    images = []
    
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))
            image = image.resize( (224, 224), Image.LANCZOS)
            image = image.convert("RGB")

            image = np.asarray(image)
            images.append(image)
        except OSError:
            pass
        
    return images

In [ ]:
# Read cat and dog data into RAM
cats = read_images( os.path.join(dataset_path, "Cat") )
dogs = read_images( os.path.join(dataset_path, "Dog") )

In [ ]:
# Change to numpy array
# @todo use all instead of first 1000 (RAM issues)
dogs = np.asarray(dogs)[:1000]
cats = np.asarray(cats)[:1000]

In [ ]:
# Append cats and dogs
x = np.concatenate([dogs, cats])
x = x.astype(np.float32)
x.shape

In [ ]:
# Add labels
y_dogs = np.zeros(len(dogs))
y_cats = np.ones(len(cats))

y = np.concatenate([y_dogs, y_cats])
y = y.reshape(-1, 1)

In [ ]:
# Scale image pixel depth
x_scaled = x / 255.

# Train a new CNN from scratch

In [ ]:
# Create CNN 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Adapt optimizer
# Solver diverges if learning rate to high!

from keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.00001), loss="binary_crossentropy", metrics=["acc"])

model.summary()

In [ ]:
# Fit model from start
# This takes forever (even on GPU up to several hours!)
model.fit(x_scaled, y, epochs=50, batch_size=64)

In [ ]:
# Free RAM
del x_scaled
del model

# Train a pre-trained VGG16 model

### Load VGG16 CNN

In [ ]:
# 
import keras.applications.vgg16 as vgg16

In [ ]:
# Preprocess data (e.g. scale)
x_vgg16 = vgg16.preprocess_input(x)

In [ ]:
# Create CNN based on VGG16 without last / top layer (output)
vgg16_model = vgg16.VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16_model.summary()

In [ ]:
# Do not optimize VGG16 weights > Takes a lot of computational time
# And these weights are already good to detect objects!
vgg16.trainable = False

### Add output layers to cropped VGG16

In [ ]:
# Create new keras CNN based on VGG16 with new output layers
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

model2 = Sequential()
model2.add(vgg16_model)

model2.add(Flatten())
model2.add(Dense(4096, activation="relu"))
model2.add(Dense(1024, activation="relu"))
model2.add(Dense(1, activation="sigmoid"))

model2.summary()

In [ ]:
# Train model
# Caution, this also takes a lot of time, as all images are run through VGG16 during
# optimization. See next model to further boost your optimization speed!!!
model2.compile(optimizer=Adam(lr=0.00001), loss="binary_crossentropy", metrics=["acc"])
model2.fit(x, y, epochs=50, batch_size=32)

### Run all images through VGG16 net and train on this data (30x boost!)
And use this data to train newly added output layers at end.

In [ ]:
# Run all x data throug VGG16 net
x_after_vgg = vgg16_model.predict(x, verbose=True)

In [ ]:
# Print shape of processes input data
x_after_vgg.shape

In [ ]:
x_after_vgg.shape

In [ ]:
# Create new keras CNN without VGG16 but using its processed images
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

model3 = Sequential()
# model2.add(vgg16_model)

model3.add(Flatten( input_shape=(7, 7, 512)))
model3.add(Dense(4096, activation="relu"))
model3.add(Dense(1024, activation="relu"))
model3.add(Dense(1, activation="sigmoid"))

model3.summary()

In [ ]:
# Shuffle data (which is sorted after cats/dogs)
# This is necessary as we use validation_split in next step
# ... which uses last 20% of data
from sklearn.utils import shuffle
x_after_vgg, y = shuffle(x_after_vgg, y)

In [ ]:
# Train model using x_after_vgg
model3.compile(optimizer=Adam(lr=0.00001), loss="binary_crossentropy", metrics=["acc"])
model3.fit(x_after_vgg, y, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
# Save model
model3.save('/tmp/transfer_learning_cats_dogs.h5')